<a href="https://colab.research.google.com/github/Ixiondbz/CSE499/blob/main/RCNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install -U plotly

import json
from pathlib import Path
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import pandas as pd
import random
import cv2

     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
from google.colab import drive
drive.mount('/drive')

folder = "MyDrive/MIDOG_Challenge" #@param {type:"string"}
midog_folder = Path("/drive") / Path(folder)

print(list(midog_folder.glob("*.*")))

Mounted at /drive
[PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.sqlite'), PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.json')]


In [ ]:
!apt-get install python3-openslide
from openslide import open_slide

In [5]:
image_folder = midog_folder / "images"

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))


annotation_file = midog_folder / "MIDOG.json"
rows = []
with open(annotation_file) as f:
    data = json.load(f)

    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])

df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])

In [12]:
!git clone --recursive https://github.com/smallcorgi/Faster-RCNN_TF.git

Cloning into 'Faster-RCNN_TF'...
remote: Enumerating objects: 406, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 406 (delta 2), reused 0 (delta 0), pack-reused 400
Receiving objects: 100% (406/406), 1.16 MiB | 8.09 MiB/s, done.
Resolving deltas: 100% (192/192), done.


In [15]:
%cd $FRCN_ROOT/lib
%make

[Errno 2] No such file or directory: '$FRCN_ROOT/lib'
/content


UsageError: Line magic function `%make` not found.


In [17]:
%cd Faster-RCNN_TF

/content/Faster-RCNN_TF


In [20]:
%cd lib

/content/Faster-RCNN_TF/lib


In [23]:
!make

python setup.py build_ext --inplace
  File "setup.py", line 84
    print extra_postargs
                       ^
SyntaxError: Missing parentheses in call to 'print'. Did you mean print(extra_postargs)?
Makefile:2: recipe for target 'all' failed
make: *** [all] Error 1


In [26]:
!ls